# **Exploração de Dados**

Exploração de dados no formato.csv, o dataset foi importado no Kaggle. Os dados contém informações sobre clientes de uma instituição financeira. Em especial, estamos interessados em explicar a segunda coluna, chamada de **default**, que indica se um cliente é **adimplente(default = 0)**, ou **inadimplente (default = 1)**, ou seja, queremos entender o porque um cliente deixa de honrar com suas dívidas baseado no comportamento de outros atributos, como salário, escolaridade e movimentação financeira. Uma descrição completa dos atributos está abaixo.

Obs: O notebook com o projeto final foi disponibilizado pela Ebac, entretanto para questão de estudos e aperfeiçoamento, reescrevi o código abaixo com base no notebook origial.



| Coluna  | Descrição |
| ------- | --------- |
| id      | Número da conta |
| default | Indica se o cliente é adimplente (0) ou inadimplente (1) |
| idade   | --- |
| sexo    | --- |
| depedentes | --- |
| escolaridade | --- |
| estado_civil | --- |
| salario_anual | Faixa do salario mensal multiplicado por 12 |
| tipo_cartao | Categoria do cartao: blue, silver, gold e platinium |
| meses_de_relacionamento | Quantidade de meses desde a abertura da conta |
| qtd_produtos | Quantidade de produtos contratados |
| iteracoes_12m | Quantidade de iteracoes com o cliente no último ano |
| meses_inatico_12m | Quantidade de meses que o cliente ficou inativo no último ano |
| limite_credito | Valor do limite do cartão de crédito |
| valor_transacoes_12m | Soma total do valor das transações no cartão de crédito no último ano |
| qtd_transacoes_12m | Quantidade total de transações no cartão de crédito no último ano |



**Lendo os dados num dataframe panda.**

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../input/ebac-python-projeto-final-dataset/Python_M10_support material.csv', na_values='na')

In [3]:
df.head(n=10)

,id,default,idade,sexo,dependentes,escolaridade,estado_civil,salario_anual,tipo_cartao,meses_de_relacionamento,qtd_produtos,iteracoes_12m,meses_inativo_12m,limite_credito,valor_transacoes_12m,qtd_transacoes_12m
0,768805383,0,45,M,3,ensino medio,casado,$60K - $80K,blue,39,5,3,1,"12.691,51","1.144,90",42
1,818770008,0,49,F,5,mestrado,solteiro,menos que $40K,blue,44,6,2,1,"8.256,96","1.291,45",33
2,713982108,0,51,M,3,mestrado,casado,$80K - $120K,blue,36,4,0,1,"3.418,56","1.887,72",20
3,769911858,0,40,F,4,ensino medio,NaN,menos que $40K,blue,34,3,1,4,"3.313,03","1.171,56",20
4,709106358,0,40,M,3,sem educacao formal,casado,$60K - $80K,blue,21,5,0,1,"4.716,22","816,08",28
5,713061558,0,44,M,2,mestrado,casado,$40K - $60K,blue,36,3,2,1,"4.010,69","1.088,07",24
6,810347208,0,51,M,4,NaN,casado,$120K +,gold,46,6,3,1,"34.516,72","1.330,87",31
7,818906208,0,32,M,0,ensino medio,NaN,$60K - $80K,silver,27,2,2,2,"29.081,49","1.538,32",36
8,710930508,0,37,M,3,sem educacao formal,solteiro,$60K - $80K,blue,36,5,0,2,"22.352,50","1.350,14",24
9,719661558,0,48,M,2,mestrado,solteiro,$80K - $120K,blue,36,6,3,3,"11.656,41","1.441,73",32


# **Estrutura**

In [4]:
df.shape # contando quantidade de e quantidade de colunas 

(10127, 16)

**Default = adimplente (0)**

In [5]:
df[df['default'] == 0].shape

(8500, 16)

**Default = inadimplente (1)**

In [6]:
df[df['default'] == 1].shape

(1627, 16)

In [7]:
qdt_total, _ = df.shape
qdt_adimplentes, _ = df[df['default'] == 0].shape
qdt_inadimplentes, _ = df[df['default'] == 1].shape

In [8]:
print(f"A proporção de clientes adimplentes é de {round(100 * qdt_adimplentes / qdt_total, 2)}%")
print(f"A proporção de clientes inadimplentes é de {round(100 * qdt_inadimplentes / qdt_total, 2)}%")

A proporção de clientes adimplentes é de 83.93%
A proporção de clientes inadimplentes é de 16.07%
